In [38]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
import vk_api

In [39]:
TOKEN = 'e059a5f4e059a5f4e059a5f4f4e34faa77ee059e059a5f4856f704b5f7931909fb8c92e'
NUMBER_OF_FRIENDS = 300
ORDER = 'random'
VERSION = '5.154'
deep = 1

In [55]:
def resolve_url(url, TOKEN):
    vk = vk_api.VkApi(token=TOKEN)
    vk = vk.get_api()
    ids = vk.utils.resolveScreenName(screen_name=url.split('/')[-1])
    return ids

In [41]:
def get_friends(user_id, COUNT, ORDER, VERSION, TOKEN):
    response = requests.get('https://api.vk.com/method/friends.get',
                            params={'access_token':TOKEN,
                                    'user_id':user_id,
                                    'order':ORDER,
                                    'count':COUNT,
                                    'v':VERSION})
    data = response.json()
    
    if 'response' in data:
        friend_ids = data['response']['items']
        return friend_ids
    else:
        return []

def get_name(TOKEN, user_id):
    vk = vk_api.VkApi(token=TOKEN)
    user = vk.method("users.get", {"user_ids": user_id})
    return user[0]['first_name'] +  ' ' + user[0]['last_name']

def get_number_of_friends(user_id, access_token):
    url = f"https://api.vk.com/method/friends.get?user_id={user_id}&access_token={access_token}&v=5.131"
    response = requests.get(url)
    data = response.json()

    if 'response' in data:
        friend_ids = data['response']['items']
        return friend_ids
    else:
        return []


def friends_deepNtree_analys(user_id, deep, TOKEN):
    friends_n_relevances = {}
    if deep < 0:
        friends_n_relevances = 'Incorrect deep'
    elif deep == 0:
        friends_n_relevances[user_id] = 1
    else:
        friends_above_n_relevances = friends_deepNtree_analys(user_id, deep - 1, TOKEN)
        friends_above_n = list(friends_above_n_relevances.keys())
        min_above_relevance = min(list(friends_above_n_relevances.values()))
        for friend_above in friends_above_n:
            friends_n = get_friends(str(friend_above), NUMBER_OF_FRIENDS, ORDER, VERSION, TOKEN)
            for n_id in friends_n:
                revelance = (friends_n.count(n_id) / len(friends_n)) * (friends_above_n_relevances[friend_above] / min_above_relevance)
                if n_id in friends_n_relevances:
                    friends_n_relevances[n_id] += revelance
                else:
                    friends_n_relevances[n_id] = revelance
    
    return friends_n_relevances

In [57]:
user_id = 122605737
num_recomendation = int(input('Enter number of recomendation: ')) + 1

Enter number of recomendation: 20


In [58]:
result = friends_deepNtree_analys(user_id, 2, TOKEN)

In [59]:
sorted_result = sorted(result.items(), key=lambda item: item[1], reverse=True)

In [60]:
friends_id = get_number_of_friends(user_id, TOKEN)

In [61]:
serise_id = []
series_name = []
i = 0
while i < num_recomendation:
    if sorted_result[i][0] not in friends_id:
        serise_id.append('vk.com/id' + str(sorted_result[i][0]))
        series_name.append(get_name(TOKEN, sorted_result[i][0]))
    else:
        num_recomendation += 1
    i += 1


In [62]:
df = pd.DataFrame({'Link':serise_id, 'Name':series_name})
df = df.iloc[1:, :]
df

,Link,Name
1,vk.com/id326975259,Danil Kravets
2,vk.com/id412953016,Alina Kazieva
3,vk.com/id307155848,Anastasia Borisenko
4,vk.com/id562399806,Inna Sol
5,vk.com/id711177060,Anastasia Kuzmicheva
6,vk.com/id414907160,Musa Kafiatullin
7,vk.com/id329463154,Pelagia Alexeeva
8,vk.com/id389981352,Platon Paxivatov
9,vk.com/id267309038,Daniil Chinyakov
10,vk.com/id518996720,Maxim Li
